# Analysis of Electoral Performance in Municipal Elections in the Metropolitan Region of Recife between 2008 and 2020

## Environment Settings

In [2]:
# Imports
import pandas as pd;
import matplotlib.pyplot as plt;

In [3]:
# Config PySpark
import os
import sys
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.getOrCreate()

In [4]:
# Creating a SparkSession
spark = SparkSession.builder.appName("LoadElectionData").getOrCreate();

## Datasets

Fontes:
* https://dadosabertos.tse.jus.br/dataset/resultados-2020/resource/977f5f4a-8d03-4b3d-aaa5-00001b996f02

2016:
* https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2016_PE.zip
* https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_partido_munzona/votacao_partido_munzona_2016.zip
* https://dadosabertos.tse.jus.br/dataset/resultados-2016/resource/ccd2564b-a576-4c17-a603-31e49f980667
* https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2016.zip
* https://cdn.tse.jus.br/estatistica/sead/odsele/perfil_eleitor_secao/perfil_eleitor_secao_2016_PE.zip

In [5]:
df_bens_candidato_16 = spark.read.csv('../datasets/bem_candidato_2016_PE.csv', encoding='latin1', sep=';', header=True, inferSchema=True);
df_perfil_16 = spark.read.csv('../datasets/perfil_eleitor_secao_2016_PE.csv', encoding='latin1', sep=';', header=True, inferSchema=True);
df_candidato_16 = spark.read.csv('../datasets/votacao_candidato_munzona_2016_PE.csv', encoding='latin1', sep=';', header=True, inferSchema=True);
df_partido_16 = spark.read.csv('../datasets/votacao_partido_munzona_2016_PE.csv', encoding='latin1', sep=';', header=True, inferSchema=True);
df_secao_16 = spark.read.csv('../datasets/votacao_secao_2016_PE.csv', encoding='latin1', sep=';', header=True, inferSchema=True);

In [6]:
df_bens_candidato_16.show(10)

+----------+-------------------+-----------+---------------+-----------------+----------+--------------------+----------+-----+-----+--------------------+------------+------------------+---------------------+---------------------+--------------------+----------------+---------------------+---------------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|SG_UF|SG_UE|               NM_UE|SQ_CANDIDATO|NR_ORDEM_CANDIDATO|CD_TIPO_BEM_CANDIDATO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|DT_ULTIMA_ATUALIZACAO|HH_ULTIMA_ATUALIZACAO|
+----------+-------------------+-----------+---------------+-----------------+----------+--------------------+----------+-----+-----+--------------------+------------+------------------+---------------------+---------------------+--------------------+----------------+---------------------+---------------------+
|18/02/2021|2024-06-16 12:06:52|       2016|              2|E

In [7]:
df_perfil_16.show(5)

+----------+-------------------+-----------+-----+------------+------------+---------------------+---------------------+-------+--------+----------------+---------+---------+---------------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------------------+--------------------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|SG_UF|CD_MUNICIPIO|NM_MUNICIPIO|CD_MUN_SIT_BIOMETRICA|DS_MUN_SIT_BIOMETRICA|NR_ZONA|NR_SECAO|NR_LOCAL_VOTACAO|CD_GENERO|DS_GENERO|CD_ESTADO_CIVIL|DS_ESTADO_CIVIL|CD_FAIXA_ETARIA|     DS_FAIXA_ETARIA|CD_GRAU_ESCOLARIDADE|DS_GRAU_ESCOLARIDADE|QT_ELEITORES_PERFIL|QT_ELEITORES_BIOMETRIA|QT_ELEITORES_DEFICIENCIA|QT_ELEITORES_INC_NM_SOCIAL|
+----------+-------------------+-----------+-----+------------+------------+---------------------+---------------------+-------+--------+----------------+---------+---------+---------------+---------------+---------------+----------------

In [8]:
df_candidato_16.show(10)

+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+------------+------------+------------+-------+--------+--------+------------+------------+--------------------+-------------------+-------------------+-----------------------+-----------------------+------------------------+------------------------+-------------+----------+----------+--------------------+------------+--------------------+-----------------------+----------------+----------------+-------------------+-----------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|NR_TURNO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|TP_ABRANGENCIA|SG_UF|SG_UE|       NM_UE|CD_MUNICIPIO|NM_MUNICIPIO|NR_ZONA|CD_CARGO|DS_CARGO|SQ_CANDIDATO|NR_CANDIDATO|        NM_CANDIDATO|  NM_URNA_CANDIDATO|NM_SOCIAL_CANDIDATO|CD_SITUACAO_CANDIDATURA|DS_SITUACAO_CANDIDATURA|CD_DETALHE_SITUACAO_CAND|DS_DETALHE_SITUACAO_CAN

In [9]:
df_partido_16.show(5)

+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+------------+------------+------------+-------+--------+--------+-------------+----------+----------+--------------------+------------+--------------------+-----------------------+-------------------+-----------------+----------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|NR_TURNO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|TP_ABRANGENCIA|SG_UF|SG_UE|       NM_UE|CD_MUNICIPIO|NM_MUNICIPIO|NR_ZONA|CD_CARGO|DS_CARGO|TP_AGREMIACAO|NR_PARTIDO|SG_PARTIDO|          NM_PARTIDO|SQ_COLIGACAO|        NM_COLIGACAO|DS_COMPOSICAO_COLIGACAO|ST_VOTO_EM_TRANSITO|QT_VOTOS_NOMINAIS|QT_VOTOS_LEGENDA|
+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+------------+------------+------------+-------+--------+-

In [10]:
df_secao_16.show(5)

+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+--------------------+------------+--------------------+-------+--------+--------+--------+----------+--------------------+--------+----------------+------------+--------------------+-------------------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|NR_TURNO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|TP_ABRANGENCIA|SG_UF|SG_UE|               NM_UE|CD_MUNICIPIO|        NM_MUNICIPIO|NR_ZONA|NR_SECAO|CD_CARGO|DS_CARGO|NR_VOTAVEL|          NM_VOTAVEL|QT_VOTOS|NR_LOCAL_VOTACAO|SQ_CANDIDATO|    NM_LOCAL_VOTACAO|DS_LOCAL_VOTACAO_ENDERECO|
+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+--------------------+------------+--------------------+-------+--------+--------+--------+----------+----------